In [1]:
import numpy as np
import pandas as pd

from pandas.io import gbq

In [5]:
import spotipy
import spotipy.util as util
spotify = spotipy.Spotify()

In [6]:
util.prompt_for_user_token("pitched@umusic.com","playlist-modify-public",client_id='a920ef611b2a499987fa4b7ae0304e43',
                           client_secret='9f19deb2a17041d6b63528f39bc07a2c',
                           redirect_uri='http://pitched-stage.umusic.net/spotify-channel.html')




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=a920ef611b2a499987fa4b7ae0304e43&response_type=code&redirect_uri=http%3A%2F%2Fpitched-stage.umusic.net%2Fspotify-channel.html&scope=playlist-modify-public in your browser


Enter the URL you were redirected to: https://pitched-stage.umusic.net/spotify-channel.html?code=AQDISQW0NiaN1nrqb-SWjNUMOVEDV4qquCLWXKk57GI5Wuz6eJd-bE18-NBEd6sGcbZc7LPfB_DiL_YxiorNGEIfVcvrrKbcpK-cAcdYeHatLI7pecWtpBm_GlWOPeolBcinuhZ9URHcpVQzX1gJ5EWPVoaTm31vdC6QxcMDlyvRyFbCts39jXGCJEC_--h697B5wYtdLqmuXKQH_QYd5bXFcIOu8y3G3lXgbiS6QdKRxtlxOm3CFhcQF4UYj4XcSpqNLA




'BQDRlzkgAN5Hs634T9H2VP_yTMjsH4y_V3TECHzwlVZ1WD0vAH6THqMe-CptyPlXIdfoAUZqQF-apfEzkJheN7uHjiOEIeeMeDmuljRK7ITGO60RNKfB1aVlF0JDHwd2UrL6S9bC8dxZPEZwTRsg7mux8f8_A9PqvLAbnHBUJFt2dw8oV6X-rw'

In [9]:
token = util.prompt_for_user_token("pitched@umusic.com","playlist-modify-public",client_id='a920ef611b2a499987fa4b7ae0304e43',
                           client_secret='9f19deb2a17041d6b63528f39bc07a2c',
                           redirect_uri='http://pitched-stage.umusic.net/spotify-channel.html')

In [11]:
sp = spotipy.Spotify(auth=token)
results = sp.search(q='upc:00600406845511', type='album')
print(results)

{'albums': {'href': 'https://api.spotify.com/v1/search?query=upc%3A00600406845511&type=album&market=GB&offset=0&limit=10', 'items': [{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'name': 'Various Artists', 'type': 'artist', 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'available_markets': ['AR', 'AT', 'AU', 'BE', 'BG', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CZ', 'DE', 'DK', 'EC', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'PA', 'PE', 'PH', 'PL', 'PT', 'PY', 'RO', 'SE', 'SG', 'SK', 'SV', 'TH', 'TR', 'TW', 'US', 'UY', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/0LcHuJ7a3BvyP8dUeopOgg'}, 'href': 'https://api.spotify.com/v1/albums/0LcHuJ7a3BvyP8dUeopOgg', 'id': '0LcHuJ7a3BvyP8dUeopOgg', 'images': [{'heigh

In [18]:
results['albums']['items'][0]['album_type']

'album'

In [19]:
get_upcs = """
SELECT
  pl.upc,
  playlist_owner,
  pl.release_date,
  release_title,
  status,
  take_down_date,
  a.album_type
FROM
  `umg-comm-tech-dev.fixed_playlists_data.playlists_list` pl
LEFT JOIN
  `umg-data-science.spotify_metadata.albums` a
USING
  (upc)
WHERE
  a.album_type IS NULL
"""

In [20]:
df = gbq.read_gbq(get_upcs, project_id='umg-comm-tech-dev', dialect='standard')

In [21]:
df.head()

,upc,playlist_owner,release_date,release_title,status,take_down_date,album_type
0,0000602567789178¬†,None,2018-06-08 00:00:00+00:00,Cottage Music Campfire Jams',None,NaT,None
1,00600406840400,Jamie Ells,2019-04-19 00:00:00+00:00,Jazz Driving Soundtrack,APPROVED,NaT,None
2,00600406844743,Savvaki Dracou,2019-07-05 00:00:00+00:00,Reggaeton Vibes,DELIVERED,NaT,None
3,00600406843364,Eve Chadbourne,2019-06-21 00:00:00+00:00,Classical Pop,APPROVED,NaT,None
4,00600406845146,Jamie Ells,2019-07-12 00:00:00+00:00,Rock Hits,APPROVED,NaT,None


In [22]:
upcs = []
types = []

for upc in df.upc:
    upcs.append(upc)
    try:
        results = sp.search(q='upc:'+upc, type='album')
        types.append(results['albums']['items'][0]['album_type'])
    except:
        types.append(np.nan)

In [23]:
df_types = pd.DataFrame(columns = ['upc','album_type'])
df_types['upc'] = upcs
df_types['album_type']=types

In [24]:
df_all = pd.merge(df, df_types, on='upc')
df_all.head()

,upc,playlist_owner,release_date,release_title,status,take_down_date,album_type_x,album_type_y
0,0000602567789178¬†,None,2018-06-08 00:00:00+00:00,Cottage Music Campfire Jams',None,NaT,None,NaN
1,00600406840400,Jamie Ells,2019-04-19 00:00:00+00:00,Jazz Driving Soundtrack,APPROVED,NaT,None,compilation
2,00600406844743,Savvaki Dracou,2019-07-05 00:00:00+00:00,Reggaeton Vibes,DELIVERED,NaT,None,compilation
3,00600406843364,Eve Chadbourne,2019-06-21 00:00:00+00:00,Classical Pop,APPROVED,NaT,None,album
4,00600406845146,Jamie Ells,2019-07-12 00:00:00+00:00,Rock Hits,APPROVED,NaT,None,album


In [25]:
df_all.to_csv('fp_upc_with_types.csv')